In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from recomendation_model.base_model import LightGBMRanker

os.chdir("c://Users//gufer//OneDrive//Documentos//FIAP//Fase_05//ML_Engineer_Datathon/")
# Carrega os dados de treino a partir dos arquivos Parquet
X_train = pd.read_parquet("data/train/X_train.parquet")
y_train = pd.read_parquet("data/train/y_train.parquet")
X_train_full = pd.read_parquet("data/train/X_train_full.parquet")
group_train = pd.read_parquet("data/train/group_train.parquet")
# Note que para o parâmetro 'group' do LightGBM, passamos os valores da coluna 'groupCount'
model = LightGBMRanker()
model.train(X_train.values, y_train.values.ravel(), group_train["groupCount"].values)

# Salve o modelo em um arquivo pickle
import pickle
with open("lightgbm_ranker.pkl", "wb") as f:
    pickle.dump(model, f)

print("Modelo treinado e salvo em 'lightgbm_ranker.pkl'")
with open("lightgbm_ranker.pkl", "rb") as f:
    model = pickle.load(f)

print("Modelo carregado com sucesso!")

Modelo treinado e salvo em 'lightgbm_ranker.pkl'
Modelo carregado com sucesso!


,userId,pageId,userType,isWeekend,dayPeriod,issuedDatetime,timestampHistoryDatetime,coldStart,localState,localRegion,...,relLocalState,relLocalRegion,relThemeMain,relThemeSub,userTypeFreq,dayPeriodFreq,localStateFreq,localRegionFreq,themeMainFreq,themeSubFreq
